In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
%matplotlib inline
import  statsmodels.api as sm
import statsmodels.stats.api as sms
from scipy.stats import boxcox

### Bringing in Happiness data

In [2]:
Happy2016 = pd.read_csv('C:/Users/singe/Downloads/2016.csv')
Happy2016.head()

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,Norway,Western Europe,4,7.498,7.421,7.575,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,Finland,Western Europe,5,7.413,7.351,7.475,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


### Keep only the columns we will be using for the analysis

In [3]:
HAPPY2016 = Happy2016[['Country', 'Happiness Rank', 'Happiness Score', 'Economy (GDP per Capita)']]
HAPPY2016.head()

,Country,Happiness Rank,Happiness Score,Economy (GDP per Capita)
0,Denmark,1,7.526,1.44178
1,Switzerland,2,7.509,1.52733
2,Iceland,3,7.501,1.42666
3,Norway,4,7.498,1.57744
4,Finland,5,7.413,1.40598


### Brining in Child Mortality Data

In [4]:
child_mortality = pd.read_csv('C:/Users/singe/Downloads/U5MR_deaths.csv')
child_mortality.head()

,ISO Code,CountryName,Uncertainty bounds*,1955.5,1956.5,1957.5,1958.5,1959.5,1960.5,1961.5,...,2009.5,2010.5,2011.5,2012.5,2013.5,2014.5,2015.5,2016.5,2017.5,2017.5.1
0,AFG,Afghanistan,Lower,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"97,607","93,455","89,155","84,664","80,132","75,600","71,242","67,382","63,692","60,178"
1,AFG,Afghanistan,Median,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"105,746","101,882","97,906","93,956","90,103","86,378","82,918","79,770","76,877","74,278"
2,AFG,Afghanistan,Upper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"114,731","111,090","107,471","104,015","100,789","97,847","95,244","92,986","91,134","89,895"
3,ALB,Albania,Lower,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,479,446,416,390,366,345,327,312,298,283
4,ALB,Albania,Median,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,517,474,438,406,380,358,340,326,313,302


### Rename some of the columns for ease of use and readability

In [5]:
child_mortality.rename(columns={'CountryName' : 'Country', 'Uncertainty bounds*' : 'Uncertainty bounds', '2016.5' : 'Child_Mortality_2016'}, inplace=True)
child_mortality.head()

,ISO Code,Country,Uncertainty bounds,1955.5,1956.5,1957.5,1958.5,1959.5,1960.5,1961.5,...,2009.5,2010.5,2011.5,2012.5,2013.5,2014.5,2015.5,Child_Mortality_2016,2017.5,2017.5.1
0,AFG,Afghanistan,Lower,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"97,607","93,455","89,155","84,664","80,132","75,600","71,242","67,382","63,692","60,178"
1,AFG,Afghanistan,Median,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"105,746","101,882","97,906","93,956","90,103","86,378","82,918","79,770","76,877","74,278"
2,AFG,Afghanistan,Upper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"114,731","111,090","107,471","104,015","100,789","97,847","95,244","92,986","91,134","89,895"
3,ALB,Albania,Lower,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,479,446,416,390,366,345,327,312,298,283
4,ALB,Albania,Median,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,517,474,438,406,380,358,340,326,313,302


### Keep only the columns we need for analysis

In [8]:
CM = child_mortality[[ 'Country', 'Uncertainty bounds', 'Child_Mortality_2016']]
CM.head()

,Country,Uncertainty bounds,Child_Mortality_2016
0,Afghanistan,Lower,"67,382"
1,Afghanistan,Median,"79,770"
2,Afghanistan,Upper,"92,986"
3,Albania,Lower,312
4,Albania,Median,326


### Select only the median value for child mortality rates. This gives us one country per row instead of 3.

In [9]:
CM3 = CM[child_mortality['Uncertainty bounds']=="Median"]
CM3.head()

,Country,Uncertainty bounds,Child_Mortality_2016
1,Afghanistan,Median,"79,770"
4,Albania,Median,326
7,Algeria,Median,"24,740"
10,Andorra,Median,2
13,Angola,Median,"98,117"


### Merge the Happiness data set with the Child Mortality Rate data set.

In [10]:
Happy22 = pd.merge(HAPPY2016, CM3)
Happy22.head()

,Country,Happiness Rank,Happiness Score,Economy (GDP per Capita),Uncertainty bounds,Child_Mortality_2016
0,Denmark,1,7.526,1.44178,Median,253
1,Switzerland,2,7.509,1.52733,Median,366
2,Iceland,3,7.501,1.42666,Median,9
3,Norway,4,7.498,1.57744,Median,157
4,Finland,5,7.413,1.40598,Median,111


### Check to see what types of data our columns are in. 

In [11]:
Happy22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 0 to 134
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Country                   135 non-null    object 
 1   Happiness Rank            135 non-null    int64  
 2   Happiness Score           135 non-null    float64
 3   Economy (GDP per Capita)  135 non-null    float64
 4   Uncertainty bounds        135 non-null    object 
 5   Child_Mortality_2016      135 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 7.4+ KB


### Change Child Mortality 2016 from object to an int. in order to run correlation matrix and further analysis.

In [12]:
Happy22.Child_Mortality_2016 = Happy22.Child_Mortality_2016.str.replace(',', '').astype(int)

##### Check to see if it worked

In [13]:
Happy22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 0 to 134
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Country                   135 non-null    object 
 1   Happiness Rank            135 non-null    int64  
 2   Happiness Score           135 non-null    float64
 3   Economy (GDP per Capita)  135 non-null    float64
 4   Uncertainty bounds        135 non-null    object 
 5   Child_Mortality_2016      135 non-null    int32  
dtypes: float64(2), int32(1), int64(1), object(2)
memory usage: 6.9+ KB


##### Great!!!

### Export new combined data set to .csv in order to use in other programs.

In [14]:
Happy22.to_csv('Happy2016', encoding='utf-8')

In [17]:
Happy22 = pd.read_csv('Happy2016', encoding='utf-8')
Happy22.head()

,Unnamed: 0,Country,Happiness Rank,Happiness Score,Economy (GDP per Capita),Uncertainty bounds,Child_Mortality_2016
0,0,Denmark,1,7.526,1.44178,Median,253
1,1,Switzerland,2,7.509,1.52733,Median,366
2,2,Iceland,3,7.501,1.42666,Median,9
3,3,Norway,4,7.498,1.57744,Median,157
4,4,Finland,5,7.413,1.40598,Median,111


### Drop the unnamed column

In [20]:
Happy222 = Happy22[[ 'Country', 'Happiness Score', 'Economy (GDP per Capita)', 'Child_Mortality_2016']]
Happy222.head()

,Country,Happiness Score,Economy (GDP per Capita),Child_Mortality_2016
0,Denmark,7.526,1.44178,253
1,Switzerland,7.509,1.52733,366
2,Iceland,7.501,1.42666,9
3,Norway,7.498,1.57744,157
4,Finland,7.413,1.40598,111


### Run Correlation Matrix

In [21]:
Happy222.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,Happiness Score,Economy (GDP per Capita),Child_Mortality_2016
Happiness Score,1.0,0.81,-0.2
Economy (GDP per Capita),0.81,1.0,-0.21
Child_Mortality_2016,-0.2,-0.21,1.0
